In [3]:
import os
import glob
import numpy as np

##########################################################
result_dir = "/Charles/project/ASm6A/peak_calling/m6Am/"
os.system("mkdir -p %s" % result_dir)
os.chdir(result_dir)
data_dir = "/home/galaxy/project/alleleSpecific_analysis/data/m6Am_inference/"
left_region = "%s/left_region_100bp.bed" % data_dir
right_region = "%s/right_region_100bp.bed" % data_dir
#
map_file = "/home/galaxy/project/alleleSpecific_analysis/data/sample_info/total_Input_vs_IP.txt"
df_map = pd.read_table(map_file)
map_dict = dict(zip(df_map['IP'], df_map['Input']))
ip_dir = "/Charles/project/ASm6A/corrected_ip/"
input_dir = "/Charles/project/ASm6A/corrected_input/"
#
m6a_dir = "/Charles/project/ASm6A/peak_calling/merged_peak_MSPC/" # CRR042278/ConsensusPeaks.bed
# m6a_dir = "/Charles/project/ASm6A/peak_calling/merged_peak_bedtools/"
# m6a_dir = "/Charles/project/ASm6A/peak_calling/macs2/10e5/"
# m6a_dir = "/Charles/project/ASm6A/peak_calling/union_peak_bedtools/"
# m6a_dir = "/Charles/project/ASm6A/peak_calling/union_three_bedtools/"
##################################################################################################

def generate_dict():
    bed = "/home/galaxy/project/alleleSpecific_analysis/data/m6Am_inference/contain_a_TSS.bed"
    tss_dict = {}
    with open(bed, 'r') as f:
        contents = f.readlines()
        for i in range(len(contents)):
            tss_dict[i] = contents[i].strip()
    return tss_dict

def select_tss(in_file):
    row_nums = []
    with open(in_file, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            input_count, ip_count = int(lines[i].split("\t")[6].strip()), int(lines[i].split("\t")[7].strip())
            if (input_count >= 5) and (ip_count >=5):
                row_nums.append(i)
    return row_nums


def count_reads():
    ip_list = glob.glob("%s/*/*-uniq.bam"%ip_dir)
    shell = "countReads.sh"
    with open(shell, 'w') as fw:
        for ip_bam in ip_list:
            prefix = os.path.basename(ip_bam).split("-uniq.bam")[0]
            input_bam = os.path.join(input_dir, "%s/%s-uniq.bam"% (map_dict[prefix], map_dict[prefix]))
            bam_line = " ".join([ip_bam, input_bam])
            left_count = os.path.join(result_dir, "%s_left_100bp_count.txt"%prefix)
            fw.write("bedtools multicov -bams %s -bed %s > %s\n" % (bam_line, left_region, left_count))
            right_count = os.path.join(result_dir, "%s_right_100bp_count.txt"%prefix)
            fw.write("bedtools multicov -bams %s -bed %s > %s\n" % (bam_line, right_region, right_count))
        
def inference_m6am():
    ip_list = glob.glob("%s/*/*-uniq.bam"%ip_dir)
    for ip_bam in ip_list:
        prefix = os.path.basename(ip_bam).split("-uniq.bam")[0]
        left_count = os.path.join(result_dir, "%s_left_100bp_count.txt"%prefix)
        right_count = os.path.join(result_dir, "%s_right_100bp_count.txt"%prefix)
        tss_dict = generate_dict()
        left_nums, right_nums = select_tss(left_count), select_tss(right_count)
        total_nums = list(set(left_nums + right_nums))
        with open("%s_selected_TSS.bed" % prefix, 'w') as fw:
            for x in total_nums:
                fw.write(tss_dict[x] + "\n")
        # ### overlap with m6a peak
        tss = "%s_selected_TSS.bed" % prefix
        m6a = os.path.join(m6a_dir, "%s/ConsensusPeaks.bed" % prefix)
        # m6a = os.path.join(m6a_dir, "macs2_%s_normalized.bed" % prefix)
        m6am = "%s_m6Am.bed"% prefix
        os.system("bedtools intersect -a %s -b %s -wa > %s" % (m6a, tss, m6am))
        # ### select m6a from raw m6a peak
        res_dir = os.path.join(m6a_dir, "rm_m6Am/")
        os.system("mkdir -p %s" % res_dir)
        #
        real_m6a = os.path.join(res_dir, "%s_m6A.bed" % prefix)
        ! grep -v -f $m6am $m6a > $real_m6a

#############
# count_reads()
# ! less countReads.sh | parallel -j15
inference_m6am()